# TINT Demonstration


This notebook will take you through the following process:

1. GET DATA - Obtain nexrad data from Amazon S3
2. GRID - Map data to cartesian grids using Py-ART
3. TRACK - Perform cell tracking using TINT
4. VISUALIZE - Visualize the tracks

In [2]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, Image, display
import tempfile
import os
os.environ['PROJ_LIB'] = r'c:\Users\CoeFamily\.conda\envs\meteorology\Library\share'
import shutil

import pyart
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate
from tint.visualization import embed_mp4_as_gif
import cartopy.crs as ccrs


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



## 1. GET DATA


In [2]:
# Let's get some nexrad data from Amazon S3 to use for tracking
# I know of some storm cells that passed over Houston back in July of 2015
keys = get_nexrad_keys('kbox', start='20020206_023000', end='20020206_073000')

Found 24 keys.


In [1]:
# We can visualize this data using pyart and matplotlib
fig = plt.figure(figsize=(10, 7))


def plot_ppi(key):
    """ Plots ppi map of nexrad data given S3 key. """
    plt.clf()
    radar = read_nexrad_key(key)
    display = pyart.graph.RadarMapDisplay(radar)
    display.plot_ppi_map('reflectivity', resolution='10m',
                         sweep=0, fig=fig,projection = ccrs.PlateCarree(),
                         lon_0=radar.longitude['data'][0],
                         lat_0=radar.latitude['data'][0])
    del display, radar


ppi_anim = FuncAnimation(fig, plot_ppi, frames=keys, interval=1000)
tmp_anim = tempfile.NamedTemporaryFile(mode='w',delete=False)
tmp_anim.close()
ppi_anim.save(tmp_anim.name + '.mp4', writer='ffmpeg')
embed_mp4_as_gif(tmp_anim.name + '.mp4')
os.remove(tmp_anim.name + '.mp4')
plt.close()

NameError: name 'plt' is not defined

## 2. GRID

### Disclaimer
If you are running this notebook and don't want this data to remain on your filesystem, make sure you run the last cell in the notebook. If your kernel crashes for some reason, you can manually remove the the path printed after the following cell.

In [4]:
# Looks like there are some interesting cells coming off the gulf.
# Let's make a function that maps these volumes to cartesian grids using pyart.
# Most of the cells seem to be in the bottom right quadrant, so we'll only grid
# that part of the domain to save time.

def get_grid(radar):
    """ Returns grid object from radar object. """
    grid = pyart.map.grid_from_radars(
        radar, grid_shape=(31, 401, 401),
        grid_limits=((0, 15000), (-200000, 0), (0, 200000)),
        fields=['reflectivity'], gridding_algo='map_gates_to_grid',
        h_factor=0., nb=0.6, bsp=1., min_radius=200.)
    return grid


# Let's write these grids to a temporary location so that we can reuse them.
# This might take a few minutes
tmp_dir = tempfile.mkdtemp()
print('temporary directory:', tmp_dir)
filenames = []
for num, key in enumerate(keys):
    print('saving grid', num)
    radar = read_nexrad_key(key)
    grid = get_grid(radar)
    name = os.path.join(tmp_dir, 'grid_' + str(num).zfill(3) + '.nc')
    filenames.append(name)
    pyart.io.write_grid(name, grid)
    del radar, grid

temporary directory: C:\Users\COEFAM~1\AppData\Local\Temp\tmpgohrwarq
saving grid 0
saving grid 1
saving grid 2
saving grid 3
saving grid 4
saving grid 5
saving grid 6
saving grid 7
saving grid 8
saving grid 9
saving grid 10
saving grid 11
saving grid 12
saving grid 13
saving grid 14
saving grid 15
saving grid 16
saving grid 17
saving grid 18
saving grid 19
saving grid 20
saving grid 21
saving grid 22
saving grid 23


In [5]:
# Now we can easily instantiate generators of these grids like so
grids = (pyart.io.read_grid(fn) for fn in filenames)

## 3. TRACK

In [6]:
# First, let's instantiate a tracks object and view the default parameters
tracks_obj = Cell_tracks()
tracks_obj.params

{'FIELD_THRESH': 20,
 'MIN_SIZE': 8,
 'SEARCH_MARGIN': 4000,
 'FLOW_MARGIN': 10000,
 'MAX_FLOW_MAG': 50,
 'MAX_DISPARITY': 999,
 'MAX_SHIFT_DISP': 15,
 'ISO_THRESH': 8,
 'ISO_SMOOTH': 3,
 'GS_ALT': 1500}

In [7]:
# The cells we're interested in look a bit small. Let's reduce the minimum size threshold.
tracks_obj.params['MIN_SIZE'] = 4
# We'll give the generator of grids we made earlier to the get_tracks method of our tracks object.
tracks_obj.get_tracks(grids)

No cells found in scan 0
No cells found in scan 1
No cells found in scan 2
Writing tracks for scan 3
No cells found in scan 4
Writing tracks for scan 5
Writing tracks for scan 6
Writing tracks for scan 7
Writing tracks for scan 8
Writing tracks for scan 9
Writing tracks for scan 10
Writing tracks for scan 11
Writing tracks for scan 12
Writing tracks for scan 13
Writing tracks for scan 14
Writing tracks for scan 15
Writing tracks for scan 16
Writing tracks for scan 17
Writing tracks for scan 18
Writing tracks for scan 19
Writing tracks for scan 20
Writing tracks for scan 21
Writing tracks for scan 22
Writing tracks for scan 23


time elapsed 0.2 minutes


In [8]:
# Now we can view the 'tracks' attribute of our tracks object to see the results.
tracks_obj.tracks.head(20)

time   grid_x   grid_y     lon     lat    area  \
scan uid                                                                     
3    0   2002-02-06 04:05:40.696   45.738  397.357  0.2068 -0.0135   10.50   
5    1   2002-02-06 04:25:24.637   60.985  388.398  0.2743 -0.0540   50.25   
     2   2002-02-06 04:25:24.637   49.400  393.775  0.2203 -0.0270   10.00   
     3   2002-02-06 04:25:24.637   28.471  395.118  0.1259 -0.0225    4.25   
6    1   2002-02-06 04:35:16.558   72.165  376.975  0.3238 -0.1034   30.25   
     3   2002-02-06 04:35:16.558   45.932  386.114  0.2068 -0.0630   11.00   
     4   2002-02-06 04:35:16.558  127.852  386.074  0.5756 -0.0630   13.50   
     5   2002-02-06 04:35:16.558   29.487  394.410  0.1304 -0.0270   29.25   
7    1   2002-02-06 04:45:08.481   87.500  359.500  0.3957 -0.1799    4.50   
     4   2002-02-06 04:45:08.481  146.038  374.962  0.6565 -0.1124   13.25   
     5   2002-02-06 04:45:08.481   36.025  390.122  0.1619 -0.0450   69.50   
     6   2002-02-06 04:45:08.481   67.783  398.174  0.3058 -0.0090    5.75   
     7   2002-02-06 04:45:08.481   95.870  398.000  0.4317 -0.0090    5.75   
8    4   2002-02-06 04:55:00.402  163.120  363.680  0.7329 -0.1619   12.50   
     8   2002-02-06 04:55:00.402   55.747  369.823  0.2518 -0.1349   19.75   
     5   2002-02-06 04:55:00.402   35.458  389.740  0.1574 -0.0450   98.25   
     9   2002-02-06 04:55:00.402  122.500  384.125  0.5486 -0.0719    4.00   
9    10  2002-02-06 05:04:52.573  113.381  347.571  0.5081 -0.2338    5.25   
     8   2002-02-06 05:04:52.573   59.741  366.704  0.2698 -0.1484   33.75   
     5   2002-02-06 05:04:52.573   34.725  388.599  0.1574 -0.0495  121.00   

             vol        max  max_alt  isolated  
scan uid                                        
3    0     6.500  24.511511      1.5     False  
5    1    25.375  27.501650      1.5     False  
     2     5.250  24.374195      1.0     False  
     3     2.125  22.227459      1.0     False  
6    1    17.250  24.688965      1.5     False  
     3     5.750  25.437498      1.0     False  
     4    12.875  25.274424      1.0     False  
     5    15.500  28.999132      1.5     False  
7    1     3.000  22.453444      1.5     False  
     4    16.750  24.967340      1.5      True  
     5    41.375  29.562500      1.5     False  
     6     2.875  22.790112      1.0     False  
     7     3.375  22.293774      1.0     False  
8    4    16.125  23.937500      1.5      True  
     8    10.625  26.405264      1.5     False  
     5    64.375  30.394611      1.5     False  
     9     4.000  21.000000      1.0     False  
9    10    5.125  23.628693      1.0     False  
     8    18.250  26.375578      1.5     False  
     5    79.625  31.681791      1.5     False

## 4. VISUALIZE

In [9]:
# We can visualize these tracks to assess the performance of the algorithm.
# First we need to make another generator of the same grids for the animator
grids = (pyart.io.read_grid(fn) for fn in filenames)

In [10]:
# Let's save the animation in our temp directory so that it will be cleaned up later.
animate(tracks_obj, grids, os.path.join(tmp_dir, 'tint_demo_anim'),
        lat_lines=np.arange(41,45, .5),
        lon_lines=np.arange(-75, -68, .5),
        tracers=True)

Animating 24 frames
Frame: 0
Frame: 1
Frame: 2
Frame: 3
Frame: 4
Frame: 5
Frame: 6
Frame: 7
Frame: 8
Frame: 9
Frame: 10
Frame: 11
Frame: 12
Frame: 13
Frame: 14
Frame: 15
Frame: 16
Frame: 17
Frame: 18
Frame: 19
Frame: 20
Frame: 21
Frame: 22
Frame: 23
Make sure ffmpeg is installed properly.


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\COEFAM~1\\AppData\\Local\\Temp\\tmpybzipasb'

In [ ]:
# The animator saved an mp4 to our local directory. Now we can embed it as a gif in this notebook.
embed_mp4_as_gif(os.path.join(tmp_dir, 'tint_demo_anim.mp4'))

In [ ]:
# Let's find the cells that were tracked for the most frames
tracks_obj.tracks.groupby(level='uid').size().sort_values(ascending=False)[:5]

In [ ]:
# Now we can view these cells from a lagrangian perspective
grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator
animate(tracks_obj, grids, os.path.join(tmp_dir, 'tint_demo_lagrangian'), style='lagrangian', uid='0', alt=2000)

In [ ]:
embed_mp4_as_gif(os.path.join(tmp_dir, 'tint_demo_lagrangian.mp4'))

In [ ]:
# We can view the attributes of this cell throughout its lifetime
tracks_obj.tracks.xs('0', level='uid')

## Clean Up

In [ ]:
# Now we can delete the directory containing all the data and output from this demo.
shutil.rmtree(tmp_dir)
print('temporary directory removed')